<a href="https://colab.research.google.com/github/JanPastorek/1-AIN-413-22-Graphs/blob/main/tutorials/Tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Isomorphisms

In [42]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import isomorphism

In [43]:
# function that takes two graphs and returns all isomorphisms
def find_isomorphisms(G1, G2):
    yield from nx.isomorphism.GraphMatcher(G1, G2).isomorphisms_iter()

# almost all graphs are non-isomorphic

In [45]:
# generate two graphs
N = 15
checked_pairs = 0
while True:
    # G1 = nx.gnp_random_graph(N, 0.5)
    G1 = nx.gnm_random_graph(N, 20)
    G2 = nx.gnm_random_graph(N, 20)
    # G2 = nx.gnp_random_graph(N, 0.5)
    isomorphisms = find_isomorphisms(G1, G2)
    checked_pairs += 1
    if checked_pairs % 100000 == 0:
        print(checked_pairs)
        break

    if len(list(find_isomorphisms(G1, G2))) > 0:
        print(list(isomorphisms))
        # draw the graphs
        nx.draw(G1, with_labels=True, font_weight='bold')
        plt.show()
        nx.draw(G2, with_labels=True, font_weight='bold')
        plt.show()
        print(nx.is_isomorphic(G1, G2))
        break

100000


# almost all graphs are asymmetric

In [46]:
import math
import itertools

In [47]:
N = 10
number_of_labelled_graphs_on_N_vertices = 2**math.comb(N,2)

n_asymmetric = 0
n_graphs_to_check = math.log2(number_of_labelled_graphs_on_N_vertices) * 100

for i in range(int(n_graphs_to_check)):
    G1 = nx.gnp_random_graph(N, 0.5)
    automorphisms = list(nx.vf2pp_all_isomorphisms(G1,G1))
    if len(automorphisms) == 1:
        n_asymmetric += 1
print(n_asymmetric, n_graphs_to_check, n_asymmetric/n_graphs_to_check)

3677 4500.0 0.8171111111111111


In [48]:
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 3s (101 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
42 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acqu

In [49]:
from manim import *
import networkx as nx
import numpy as np


In [50]:
import itertools

In [52]:
%%manim -qm -v WARNING IsomorphismAndCounterexample

def npify(p): return np.array([p[0], p[1], 0.0])

def make_outside_labels(graph_mobj: "Graph", layout: dict, font_size=28, pad=0.32):
    """
    Places labels just outside each vertex, pushed outward from the graph's centroid.
    Returns a VGroup and a dict node->label mobject.
    """
    pts = np.array([npify(layout[k]) for k in layout.keys()])
    centroid = pts.mean(axis=0)
    labels = {}
    group = VGroup()
    for node, pos in layout.items():
        pos = npify(pos)
        direction = pos - centroid
        if np.linalg.norm(direction) < 1e-6:
            direction = np.array([1.0, 0.0, 0.0])
        direction = direction / np.linalg.norm(direction)
        lbl = Text(str(node), font_size=font_size)
        lbl.move_to(pos + direction * pad)
        labels[node] = lbl
        group.add(lbl)
    return group, labels

class IsomorphismAndCounterexample(Scene):
    def construct(self):
        # --- Two small isomorphic graphs ---
        # G1: 4-cycle with a diagonal (0-2)
        G1 = nx.Graph()
        G1.add_edges_from([(0,1),(1,2),(2,3),(3,0),(0,2)])

        # G2: relabeled; diagonal (b-d)
        G2 = nx.Graph()
        G2.add_edges_from([("a","b"),("b","c"),("c","d"),("d","a"),("b","d")])

        # --- Layouts (spread left/right) ---
        pos1 = nx.circular_layout(G1, scale=1.8)
        pos2 = nx.circular_layout(G2, scale=1.8)
        layout_left  = {k: npify(p)+LEFT*3.7  for k,p in pos1.items()}
        layout_right = {k: npify(p)+RIGHT*3.7 for k,p in pos2.items()}

        # --- Manim Graphs (no built-in labels: we’ll add our own outside) ---
        g1 = Graph(
            list(G1.nodes), list(G1.edges),
            layout=layout_left,
            vertex_config={"radius":0.16, "fill_color":WHITE, "stroke_width":3},
            edge_config={"stroke_width":3},
        )
        g2 = Graph(
            list(G2.nodes), list(G2.edges),
            layout=layout_right,
            vertex_config={"radius":0.16, "fill_color":WHITE, "stroke_width":3},
            edge_config={"stroke_width":3},
        )

        title = Text("Graph Isomorphism: a proof and a counterexample", font_size=36).to_edge(UP)
        self.play(FadeIn(title))
        self.play(Create(g1), Create(g2))

        # --- Always-visible outside labels ---
        g1_label_group, g1_labels = make_outside_labels(g1, layout_left, font_size=30, pad=0.34)
        g2_label_group, g2_labels = make_outside_labels(g2, layout_right, font_size=30, pad=0.34)
        self.play(FadeIn(g1_label_group), FadeIn(g2_label_group))
        self.wait(0.4)

        # --- Helper: check a mapping; show ✓/✗ with a concrete pair that (mis)behaves ---
        def demo_mapping(mapping: dict, heading: str):
            header = Text(heading, font_size=28, color=YELLOW).next_to(title, DOWN)
            self.play(FadeIn(header))

            colors = [YELLOW, TEAL, ORANGE, PURPLE, BLUE, PINK]
            c_by_u = {u: colors[i % len(colors)] for i, u in enumerate(mapping.keys())}

            # color corresponding vertices
            self.play(
                *[g1[u].animate.set_fill(c_by_u[u], opacity=1.0) for u in mapping.keys()],
                *[g2[mapping[u]].animate.set_fill(c_by_u[u], opacity=1.0) for u in mapping.keys()],
                run_time=4
            )

            # quick certificate: find one mismatched pair if any; else show all edges preserved + a sampled non-edge
            bad_pair = None
            for u,v in itertools.combinations(G1.nodes, 2):
                a1 = G1.has_edge(u,v)
                mapped_u, mapped_v = mapping[u], mapping[v]
                # Normalize the edge representation for undirected graphs
                mapped_edge_key = tuple(sorted((mapped_u, mapped_v)))
                a2 = G2.has_edge(mapped_u, mapped_v)

                if a1 != a2:
                    bad_pair = (u,v,a1,a2, mapped_u, mapped_v)
                    break

            def arrow_pair(u,v, color_u=YELLOW, color_v=TEAL):
                a1 = Arrow(g1[u].get_center(), g2[mapping[u]].get_center(), buff=0.22, stroke_width=3, tip_length=0.14)
                a2 = Arrow(g1[v].get_center(), g2[mapping[v]].get_center(), buff=0.22, stroke_width=3, tip_length=0.14)
                a1.set_color(color_u); a2.set_color(color_v)
                return a1, a2

            if bad_pair is not None:
                u,v,a1_edge,a2_edge, mapped_u, mapped_v = bad_pair
                aU, aV = arrow_pair(u,v, c_by_u[u], c_by_u[v])
                self.play(Create(aU), Create(aV), run_time=3)

                # highlight the edge/non-edge status in each graph
                # In G1:
                highlight1 = VGroup(g1[u], g1[v], g1.edges[u,v] if G1.has_edge(u,v) else VGroup())
                # Normalize the edge key for g2
                mapped_edge_key = tuple(sorted((mapped_u, mapped_v)))
                print(f"Mapped edge key: {mapped_edge_key}") # Debugging print
                highlight2 = VGroup(g2[mapped_u], g2[mapped_v], g2.edges[mapped_edge_key] if G2.has_edge(mapped_u, mapped_v) else VGroup())

                # Tick/Cross
                expl = Tex(
                    f"$\\times$ Mismatch: ({u},{v}) is {'an edge' if a1_edge else 'not an edge'} in $G_1$",
                    f"but ({mapped_u},{mapped_v}) is {'an edge' if a2_edge else 'not an edge'} in $G_2$.", color=RED
                ).scale(0.6).to_edge(DOWN)
                self.play(Write(expl), run_time=4)
                self.wait(3)
                self.play(FadeOut(aU), FadeOut(aV), FadeOut(expl))
            else:
                # Show that edges map to edges
                edge_header = Text("Edges preserved", font_size=24).next_to(header, DOWN)
                self.play(FadeIn(edge_header))
                for (u,v) in G1.edges():
                    aU, aV = arrow_pair(u,v, c_by_u[u], c_by_u[v])
                    mapped_u, mapped_v = mapping[u], mapping[v]
                     # Normalize the edge key for g2
                    mapped_edge_key = tuple(sorted((mapped_u, mapped_v)))
                    print(f"Mapped edge key: {mapped_edge_key}") # Debugging print
                    self.play(
                        g1[u].animate.set_stroke(color=c_by_u[u], width=6),
                        g1[v].animate.set_stroke(color=c_by_u[v], width=6),
                        Create(aU), Create(aV),
                        Indicate(g1.edges[u,v], scale_factor=1.06),
                        Indicate(g2.edges[mapped_edge_key], scale_factor=1.06),
                        run_time=3
                    )
                    self.play(FadeOut(aU), FadeOut(aV))
                self.play(FadeOut(edge_header))

                # Show a sampled non-edge also maps to non-edge
                non_edges = list(nx.non_edges(G1))
                if non_edges:
                    u,v = non_edges[0]
                    aU, aV = arrow_pair(u,v, c_by_u[u], c_by_u[v])
                    mapped_u, mapped_v = mapping[u], mapping[v]
                    expl = Tex(
                        f"$\\checkmark$ Non-adjacency preserved: ({u},{v}) not an edge in $G_1$",
                        f"and ({mapped_u},{mapped_v}) not an edge in $G_2$.",
                    ).scale(0.6).to_edge(DOWN)
                    self.play(Write(expl), run_time=3)
                    self.wait(2)
                    self.play(FadeOut(aU), FadeOut(aV), FadeOut(expl))

                # Formal summary
                summary = MathTex(
                    r"\forall u\neq v:\;[\,u\sim v\iff f(u)\sim f(v)\,]\;\Rightarrow\;f\text{ is an isomorphism.}"
                ).scale(0.7).to_edge(DOWN)
                summary.set_color(GREEN)
                self.play(Write(summary))
                self.wait(3)
                self.play(FadeOut(summary))

            # reset colors
            self.play(
                *[g1[u].animate.set_fill(WHITE) for u in mapping.keys()],
                *[g2[mapping[u]].animate.set_fill(WHITE) for u in mapping.keys()],
                *[g1[u].animate.set_stroke(WHITE, width=3) for u in mapping.keys()],
                *[g2[mapping[u]].animate.set_stroke(WHITE, width=3) for u in mapping.keys()],
                FadeOut(header),
                run_time=4
            )

        # --- Good isomorphism: f_good proves G1 ≅ G2 ---
        f_good = {0:"b", 1:"c", 2:"d", 3:"a"}   # bijection that works
        demo_mapping(f_good, "Exhibit a bijection f: V(G₁)→V(G₂) and verify adjacency & non-adjacency")

        # --- Bad mapping: f_bad breaks adjacency preservation (counterexample) ---
        # Here we purposely swap two images to break preservation.
        f_bad = {0:"a", 1:"c", 2:"d", 3:"b"}    # NOT an isomorphism
        demo_mapping(f_bad, "Counterexample: this mapping fails the isomorphism condition")


Manim Community v0.19.0

Mapped edge key: ('b', 'c')


Mapped edge key: ('a', 'b')


Mapped edge key: ('b', 'd')


Mapped edge key: ('c', 'd')


Mapped edge key: ('a', 'd')


Mapped edge key: ('a', 'c')
